# December 17, 2021

https://adventofcode.com/2021/day/17

In [1]:
import pandas as pd
import numpy as np

In [ ]:
data = {"x":[20,30], "y":[-10,-5]} # read from file
test = {"x":[20,30], "y":[-10,-5]}

# Part 1

This may be solved analytically. We assume ymax < 0.

To hit the highest peak, we want to choose the largest y0 that hits the target.

For y0 > 0, the y-deltas are  
y0, y0-1, ..., 1, 0, -1, ... -y0, at which point the probe is back were it started.  
Thus, pos.y( y0, 2y0+1) = 0

The first point below 0 has y = -(y0+1).  
If y0 > abs(ymin), then this point has y < ymin.  
If y0 = abs(ymin)-1, then the point has y = ymin.

The peak for this point is y0(y0+1)/2

In [ ]:
def part1(data):
    y0 = -data["y"][0]-1
    return int( y0*(y0+1)/2 )

In [ ]:
part1(test)

In [ ]:
part1(data)

# Part 2

After doing something more convoluted incorrectly for a while, I determined the simplest strategy.

For each t, find the ranges for x0 and y0 that are in the target at time t.  
The smallest t is 1, if we fire directly into the target in 1 step.  
The largest t comes from firing the probe UP as high as possible.  
By Part1 logic, this means y0=abs(ymin)-1.  
This shot reaches the target at time 2y0+2 = twice abs(ymin).

Note that a certain range of x0s will lose all forward momentum within the target. These work for sufficiently large t.

In [ ]:
def find_stoppers(xmin, xmax):
    '''which values of x0 will stop forward momentum inside the target?'''
    
    # For x0, the foreward momentum stops at x0 + x0-1 + x0-2 = x0*(x0+1)/2
    # Solve the quadratic for xmin <= x0*(x0+1)/2 <= xmax
    xlo = int( np.ceil(single_pos_root(.5, .5, -xmin)) )
    xhi = int( np.floor(single_pos_root(.5, .5, -xmax)) )
    return xlo, xhi

# I am the very model of a modern major general
def root(a,b,c):
    '''quadratic equation solver'''
    det = (b**2 - 4*a*c)
    if det < 0:
        return []
    
    x1 = (-b + det**.5) / (2*a)

    if det == 0:
        return [x1]
    
    return [x1, (-b - det**.5) / (2*x)]

def pos_roots(a,b,c):
    '''return just the positive roots'''
    return [r for r in root(a,b,c) if r > 0]

def single_pos_root(a,b,c):
    '''Given the problem statement, there should be exactly one positive root'''

    t = pos_roots(a,b,c)
    if len(t) != 1:
        raise Exception("Expected exactly one positive root!")
    return t[0]

In [ ]:
def solve_for_exlim( t, xmin, xmax ):
    '''find range of x0s that are inside [xmin,xmax] at time t'''

    # if t > smallest stopping x0, then it works for all stoppers
    # this is because faster x0s must be to the right of that x0 and by defn they never go past xmax

    # I haven't proven that there can't be faster non-stopping x0s that also work, but this seems to solve the problem.
    stop_lim = find_stoppers(xmin, xmax)
    if t >= stop_lim[0]:
        xlo = stop_lim[0]
        xhi = stop_lim[1]
    else:
        # For smaller t, find integer solutions to
        # xmin <= xt - (t-1)t/2 <= xmax
        xlo - int( np.ceil(xmin/t + (t-1)/2) )
        xhi = int( np.floor(xmax/t + (t-1)/2) )
    return xlo, xhi

def solve_for_ylim( t, ymin, ymax ):
    '''find range of y0s that are inside [ymin,ymax] at time t'''
    # Find integer solutions to
    # ymin <= yt - (t-1)t/2 <= ymax
    ylo = int( np.ceil(ymin/t + (t-1)/2) )
    yhi = int( np.floor(ymax/t + (t-1)/2) )
    return ylo, yhi

def find_xy_by_t( data ):
    # Problem specs
    xmin, xmax = data["x"]
    ymin, ymax = data["y"]

    # Range of times that have solutions
    tmin = 1
    tmax = 2*abs(ymin)

    aims = []
    for t in range(tmin, tmax+1):
        # For each t, get the xs and ys.
        xlo, xhi = solve_for_xlim(t, xmin, xmax)
        ylo, yhi = solve_for_ylim(t, ymin, ymax)

        # the same (x,y) may work for multiple times,
        # so check for unique-ness before appending
        for x in range(xlo, xhi+1):
            for y in range(ylo, yhi+1):
                if (x,y) not in aims:
                    aims.append( (x,y) )
    
    return aims


In [ ]:
ans = find_xy_by_t( test )
len(ans)

In [ ]:
ans = find_xy_by_t( data )
len(ans)